Сегодня мы поговорим еще раз про разные структуры графов. Подчиняются ли реальные графы каким либо законам?

Начнем традиционно с загрузки.

In [ ]:
import json
full_friends = json.load(open("nodes.txt", "tr"))
full_graph = json.load(open("edges.txt", "tr"))
import networkx as nx
G = nx.Graph()
for i in full_friends["items"]:
    G.add_node(i["id"], name = i["first_name"]+" "+i["last_name"], sex = i["sex"])
my_friends = list(nx.nodes(G))
for i in my_friends:
    if "items" in full_graph[str(i)]:
        for j in full_graph[str(i)]["items"]:
            if j in my_friends:
                G.add_edge(i, j)
                


Посмотрим на распределение степеней в моем графе:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import collections
from scipy.special import factorial, zeta



degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
mean_degree = np.mean(degree_sequence)
cnt = len(degree_sequence)
t = np.arange(1, 130, 1)
poiss = cnt*np.exp(-mean_degree)*np.power(mean_degree, t)/factorial(t)
l=2.0
power = cnt*np.power(t, -l)/zeta(l)


degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
plt.plot(deg, cnt, "ro")
plt.plot(deg, cnt)
plt.plot(t,poiss)
plt.plot(t[1:], power[1:])


In [ ]:
from math import log
deg_l = [log(x+1) for x in deg]
cnt_l = [log(x) for x in cnt]

poiss_l = [log(x) for x in poiss]
power_l = [log(x) for x in power]
t_l = [log(x) for x in t]


plt.plot(deg_l, cnt_l, "ro")
plt.plot(deg_l, cnt_l)
plt.plot(t_l[:40], poiss_l[:40])
plt.plot(t_l, power_l)

И так мы видим, что наше распределение не совсем пуассоновское, как следовало бы ожидать при случайном графе. Но при этом мы не видим и так называемого Power Law. Power law это предположение что распределение степеней вершин пропорционально $k^{-\lambda}$.

Вместе с тем, глядя на логарифмический график, мы хорошо видим, что крупные хабы очень сильно выбиваются из пуассоновского распределения, и немного меньше из Power law.

Например, для графа описывающего фрагмент интернета мы вмдим следующую картинку:![](http://networksciencebook.com/images/ch-04/figure-4-2.jpg)




Собственно степенной закон встречается не только в больших графах. Классическим примером степенного закона является принцип Парето. В качестве развлечения попробуйте найти статистику по доходам россиян и посмотреть на ее распределение.

Графы распределение вершин в которых ведет себя как степенное распределение мы будем называть scale-free? d heccrjv bcgjkmpe.n вариант безмасштабная сеть или масштабно-инвариантная сеть.

Главная особенность степенного распределения заключается в том, что при $\lambda < 3$ дисперсия становится бесконечной, а это приводит к возможности <<черных лебедей>>. То есть существование ютуб канала T-Series с 160+ млн. подписчиков или PewDiePie c 107 это вполне соответствует модели.

Возьмем еще пример случаного графа из кники Барабаси: ![](http://networksciencebook.com/images/ch-04/figure-4-4.jpg)

In [ ]:
                
def plot_graph(g, coloring = [], palette = plt.cm.Set2):
    with plt.xkcd():
        k = nx.degree(g)
#        print(k)
        plt.figure(1, figsize=(30,20))
        coord = nx.kamada_kawai_layout(g)
#        labels={nd: g.node[nd]['name'] for (nd) in g.nodes()}
        if len(coloring)>0:
            nx.draw_networkx(g, pos=coord, nodelist=[x[0] for x in k], node_size=[v[1]*50 for v in k], 
                         font_size=17, node_color=coloring, cmap=palette)
        else:
            nx.draw_networkx(g, pos=coord, nodelist=[x[0] for x in k], node_size=[v[1]*50 for v in k], 
                         font_size=17)

In [ ]:
G_sf = nx.scale_free_graph(len(nx.nodes(G)))
plot_graph(G_sf)

In [ ]:
degree_sequence = sorted([d for n, d in G_sf.degree()], reverse=True)  # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
plt.plot(deg, cnt, "ro")
plt.plot(deg, cnt)
plt.plot(t,poiss)
plt.plot(t[1:], power[1:])


In [ ]:
deg_l = [log(x+1) for x in deg]
cnt_l = [log(x) for x in cnt]

poiss_l = [log(x) for x in poiss]
power_l = [log(x) for x in power]
t_l = [log(x) for x in t]


plt.plot(deg_l, cnt_l, "ro")
plt.plot(deg_l, cnt_l)
plt.plot(t_l[:40], poiss_l[:40])
plt.plot(t_l, power_l)

Одна из больших проблем, с которой мы сталкиваемся в случае scale-free сетей, это очень большой разброс степеней. И если по небольшому фрагменту сети подчиняющейся пуассоновскому распределению, мы понимаем что в ней может быть, а чего нет, то в scale-free это не так.  

![](http://networksciencebook.com/images/ch-04/figure-4-8.jpg)

Существование крупных хабов приводит к тому, что среднее расстояние в таких сетях существенно сокращаются.

![](http://networksciencebook.com/images/ch-04/figure-4-12.jpg)

![](http://networksciencebook.com/images/ch-04/figure-box-4-5.jpg)

Мы поговорили про особенности scale-free сетей. На самом деле сейчас периодически возникают споры, насколько все же реальные сети соответствуют степенному закону распределения степеней вершин. Но очень часто это оказывается достаточно хорошим и разумным предположением.

Следует понимать, что часто эффект от такого предположения будет достигаться на достаточно больших сетях.


В попытках объяснить формирование реальных scale-free сетей и объяснить наличие крупных хабов или центров притяжения была предпринята попытка изучить не только сам результат и текущее состояние сети, но и ее эволюцию. Почему сеть становится именно такой. Что к этому приводит?

Одна из идей заключается в понимании, что в обычной случайной сети у ребра нет предпочтений. И у узла тоже нет. Ему все равно к чему присоединиться, лишь бы обеспечить себе необходимое число связей.

Например, если у нас задано распределение степеней, то мы можем генерировать граф с заданным распределением по простой схеме. На каждом шаге выбираем две вершины (в некоторых моделях не обязательно разные) с вероятностью пропорциональной <<еще не обеспеченной степени вершины>>. 

Но что делать если мы не знаем истинных степеней вершин, но хотим обеспечить похожую генерацю графа? Ну мы можем предположить, что <<необеспеченная степень>> сравнима с уже собранной. То есть новые ребра (исходящие преимущественно из новых узлов) будут выбирать уже существующие узлы пропорционально их степени. Такой процесс называется *предпочтительное присоединение* (Preferential Attachment).

Крупные хабы становятся <<известными>> и их известность привлекает новых соседей.


А вот краткая история по версии Барабаси:
![](http://networksciencebook.com/images/ch-05/figure-5-2.jpg)




Формально: 

Путь уже есть сеть с $n$ узлами со степенями $k_i$.
Тогда новый узел прикрепляется к $i$-тому узлу с вероятностью $\frac{k_i}{\sum k_i}$. В некоторых обобщениях узел может присоединяться и к самому себе и к узлам с которыми он уже соединен. Это происходит если число новых связей у новоого узла больше одной.

Ваша задача реализовать модель Барабаси-Альберт и посмотреть на распределение степеней вершин.
